In [ ]:
import openai
import os

from dotenv import find_dotenv, load_dotenv

from langsmith import RunTree, utils


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

MODEL_NAME = os.environ["MODEL_NAME"]

os.environ["LANGCHAIN_TRACING_V2"] = "false"
utils.tracing_is_enabled()	# this should return false

False

In [ ]:
import nest_asyncio

from openai import OpenAI
from typing import Dict, List
from utils import get_vector_db_retriever


openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


def retrieve_documents(parent_run: RunTree, question: str):
    # create a child run
    child_run = parent_run.create_child(
        name="Retrieve Documents",
        run_type="retriever",
        inputs={"question": question},
    )
    documents = retriever.invoke(question)
    # post the output of our child run
    child_run.end(outputs={"documents": documents})
    child_run.post()
    return documents


def generate_response(parent_run: RunTree, question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """
		You are an assistant for question-answering tasks. 
		Use the following pieces of retrieved context to answer the latest question in the conversation. 
		If you don't know the answer, just say that you don't know. 
		Use three sentences maximum and keep the answer concise.
    """
    # create a child run
    child_run = parent_run.create_child(
        name="Generate Response",
        run_type="chain",
        inputs={
            "question": question, 
            "documents": documents
        },
    )
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    openai_response = call_openai(child_run, messages)
    # post the output of our child run
    child_run.end(outputs={"openai_response": openai_response})
    child_run.post()
    return openai_response


def call_openai(parent_run: RunTree, messages: List[Dict], model: str = MODEL_NAME, temperature: float = 0.0) -> str:
    # create a child run
    child_run = parent_run.create_child(
        name="OpenAI Call",
        run_type="llm",
        inputs={"messages": messages},
    )
    openai_response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    # post the output of our child run
    child_run.end(outputs={"openai_response": openai_response})
    child_run.post()
    return openai_response


def langsmith_rag(question: str):
    # create a root RunTree
    root_run_tree = RunTree(
        name="Chat Pipeline",
        run_type="chain",
        inputs={"question": question}
    )

    # pass our RunTree into the nested function calls
    documents = retrieve_documents(root_run_tree, question)
    response = generate_response(root_run_tree, question, documents)
    output = response.choices[0].message.content

    # post our final output
    root_run_tree.end(outputs={"generation": output})
    root_run_tree.post()
    return output

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
question = "How can I trace with RunTree?"
ai_answer = langsmith_rag(question)
ai_answer

"You can trace with RunTree by using the RunTree API, which allows you to manually create runs and child runs to assemble your trace. You need to set your LANGSMITH_API_KEY, and you can convert LangChain's RunnableConfig to a RunTree object using RunTree.fromRunnableConfig. Additionally, you can use the traceable function to create traceable runs within your application."